In [1]:
%%bash 
date 

ls -la 

pwd

python.exe --version 

ollama.exe list 


Fri Mar 21 13:47:59 EDT 2025
total 40
drwxrwxrwx 1 jmake jmake   512 Mar 21 13:47 .
drwxrwxrwx 1 jmake jmake   512 Mar 20 20:38 ..
drwxrwxrwx 1 jmake jmake   512 Mar 21 13:16 .ipynb_checkpoints
drwxrwxrwx 1 jmake jmake   512 Mar 20 20:56 ChromaDB
drwxrwxrwx 1 jmake jmake   512 Mar 20 20:45 Data
-rwxrwxrwx 1 jmake jmake 14409 Mar 21 13:47 agentic_rag_1a1.ipynb
-rwxrwxrwx 1 jmake jmake     4 Mar 20 20:57 app_4.py
-rwxrwxrwx 1 jmake jmake 20055 Mar 21 13:31 executed_notebook.ipynb
-rwxrwxrwx 1 jmake jmake  2187 Mar 20 20:40 ingest_pdfs.py
/mnt/d/z2025_1/AI/RAGs/Test4
Python 3.12.6


In [2]:
%%time 
## Created '6709' chunks from 'Data/Blender43Manual.pdf' at './ChromaDB'
#!python.exe ingest_pdfs.py 

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time 
## python.exe -m pip install langchain_ollama
import smolagents
from langchain_ollama.llms import OllamaLLM

hf_token = ""
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

engine = smolagents.HfApiModel(model_id=model_id, token=hf_token, max_tokens=5000)
print( engine.to_dict() )

messages = [{"role": "user", "content": [{"type": "text", "text": "Hello, how are you?"}]}]

response = engine(messages, stop_sequences=["END"])
print(response.content)


D:\z2025_1\AI\RAGs\Env4\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'max_tokens': 5000, 'last_input_token_count': None, 'last_output_token_count': None, 'model_id': 'Qwen/Qwen2.5-Coder-32B-Instruct', 'provider': None}
Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
CPU times: total: 1.75 s
Wall time: 2.3 s


In [4]:
%%time
import smolagents

# Create the reasoner for better RAG
reasoner = smolagents.CodeAgent(tools=[], model=engine, add_base_tools=False, max_steps=2)
print( list(reasoner.to_dict().keys()) )


This agent has step_callbacks: they will be ignored by this method. 1

['tools', 'model', 'managed_agents', 'prompt_templates', 'max_steps', 'verbosity_level', 'grammar', 'planning_interval', 'name', 'description', 'requirements', 'authorized_imports', 'executor_type', 'executor_kwargs', 'max_print_outputs_length']
CPU times: total: 93.8 ms
Wall time: 91 ms


In [5]:
%%time 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

def load_and_process_pdfs(data_dir: str):
    """Load PDFs from directory and split into chunks."""
    loader = DirectoryLoader(
        data_dir,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

data_dir = "Data"
chunks = load_and_process_pdfs(data_dir)


CPU times: total: 45.2 s
Wall time: 45.6 s


In [6]:
## python.exe -m pip install rank_bm25
import smolagents
from langchain_community.retrievers import BM25Retriever

class RetrieverTool(smolagents.Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

retriever_tool = RetrieverTool(chunks)
print( retriever_tool )


In [7]:
%%time
import smolagents

agent = smolagents.CodeAgent(tools=[retriever_tool], model=engine, max_steps=2, verbosity_level=0)
print( agent )

CPU times: total: 31.2 ms
Wall time: 15.8 ms


In [8]:
%%time 
query = "List the Principled BSDF Inputs"  
agent_output = agent.run(query)
print(f"Final output: '{agent_output}' ") ## 22

Final output: '['Base Color', 'Subsurface', 'Subsurface Radius', 'Subsurface Color', 'Metallic', 'Specular', 'Specular Tint', 'Roughness', 'Anisotropic', 'Anisotropic Rotation', 'Sheen', 'Sheen Tint', 'Clearcoat', 'Clearcoat Roughness', 'IOR', 'Transmission', 'Transmission Roughness', 'Normal', 'Clearcoat Normal', 'Tangent']' 
CPU times: total: 2.02 s
Wall time: 21.1 s


In [9]:
%%time 
query = "Waht is the meaning of BSDF?"
agent_output = agent.run(query)
print(f"Final output: '{agent_output}' ")  ## Bidirectional Scattering Distribution Function

Final output: 'BSDF stands for Bidirectional Scattering Distribution Function and describes how light interacts with surface materials in rendering.' 
CPU times: total: 31.2 ms
Wall time: 14.5 s


In [10]:
%%bash 
## jupyter-nbconvert.exe --to notebook --execute .\agentic_rag_1a1.ipynb --output executed_notebook.ipynb
date 

Fri Mar 21 13:49:23 EDT 2025
